In [43]:
from osgeo import gdal, osr
from osgeo import osr
import geopandas as gpd
from osgeo import ogr
from shapely.geometry import Polygon, Point
from geopandas.geoseries import *
import re
import tarfile
import pandas as pd
import datetime as dt
import os
import glob
import subprocess
import os.path
from os import path
import json
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from terrautils.spatial import scanalyzer_to_latlon

In [44]:
# Grabs the thermal raw data tar file for the Gantry's metadata for the images taken at the specified date
class JSON:
    def get_tar(season, date):
        command = f'iget -rKTPf -N 0 /iplant/home/shared/terraref/ua-mac/raw_tars/{season}/flirIrCamera/flirIrCamera-{date}.tar'
        subprocess.call(command, shell = True)
        command = f'tar -xvf flirIrCamera-{date}.tar'
        subprocess.call(command, shell = True)

# Finds the individual json files and adds them to the filepath (end up with a list of paths for each individual json file)
    def pathlist(season, date):
        json_data = JSON.get_tar(season, date)
        pathlist = Path(f"./flirIrCamera/{date}/").glob('**/*.json')
        JSON_path_list = []
        for path in pathlist:
            path_str = str(path)
            JSON_path_list.append(path_str)
        return JSON_path_list

# Uses the pathlists and searches each one
# Gathers the time, image name, gantry_x and gantry_y information from the metadata of each individual image and adds to dictionary
    def time_dict(season, date):
        file_path_list = JSON.pathlist(season, date)
        JSON_dict = dict()
        for file in file_path_list:
            path_metadata = glob.glob(f'{file}')
            metadata = str(path_metadata)[2:-2]
            with open(metadata) as f:
                meta = json.load(f)['lemnatec_measurement_metadata']
                time = (meta['gantry_system_variable_metadata']['time'])
                gantry_x = float(meta['gantry_system_variable_metadata']['position x [m]'])
                gantry_y = float(meta['gantry_system_variable_metadata']['position y [m]'])
                gantry_z = float(meta['gantry_system_variable_metadata']['position z [m]'])
                filename = os.path.basename(metadata)
            if JSON is not JSON_dict:
                JSON_dict[time, filename, gantry_x, gantry_y, gantry_z] = "Date, Time, Gantry_x, Gantry_y, Gantry_z"
            else:
                print("JSON already in Dictionary")
        return sorted(JSON_dict)

# Searches through the dictionary created and creates a dataframe of the information in the dictionary
    def time_df(season, date):
        JSON_time_d = JSON.time_dict(season, date)
        JSON_time_df = pd.DataFrame.from_dict(JSON_time_d)
        JSON_time_df.columns = ['Date and Time', 'Image Name', 'Gantry_x', 'Gantry_y', "Gantry_z"]
        return JSON_time_df

# Converts the gantry coordinates into GPS coordinates (using 'terrautils')
# Used when we defined an image as a Point
    def GPS_coord (season, date):
        data = JSON.time_df(season, date)
        gantry_x_pos = data['Gantry_x']
        gantry_y_pos = data['Gantry_y']        
        GPS_latlon = scanalyzer_to_latlon(gantry_x_pos, gantry_y_pos)
        GPS_df = pd.DataFrame(GPS_latlon)
        GPS_latlon_df = GPS_df.transpose()
        GPS_latlon_df.columns = ['GPS_lat', 'GPS_lon']
        data['GPS_lat'] = GPS_latlon_df['GPS_lat']
        data['GPS_lon'] = GPS_latlon_df['GPS_lon']
        return data
    
# Takes Metadata for the image and creates a bounding box
# Used when we defined image as a polygon rather than using the center point
    def b_box(season, date):
        file_path_list = JSON.pathlist(season, date)
        JSON_dict = dict()
        for file in file_path_list:
            path_metadata = glob.glob(f'{file}')
            metadata = str(path_metadata)[2:-2]
            with open(metadata) as f:
                meta = json.load(f)['lemnatec_measurement_metadata']
                time = (meta['gantry_system_variable_metadata']['time'])
                gantry_x = float(meta['gantry_system_variable_metadata']['position x [m]'])
                gantry_y = float(meta['gantry_system_variable_metadata']['position y [m]'])
                gantry_z = float(meta['gantry_system_variable_metadata']['position z [m]'])
                fov_x, fov_y = float(meta['sensor_fixed_metadata']['field of view x [m]']), float(meta['sensor_fixed_metadata']['field of view y [m]'])
                B = gantry_z
                A_x = np.arctan((0.5*float(fov_x))/2)
                A_y = np.arctan((0.5*float(fov_y))/2)
                L_x = 2*B*np.tan(A_x)
                L_y = 2*B*np.tan(A_y)
                x_n = gantry_x + (L_x/2)
                x_s = gantry_x - (L_x/2)
                y_w = gantry_y + (L_y/2)
                y_e = gantry_y - (L_y/2)
                bbox_nw_latlon = scanalyzer_to_latlon(x_n, y_w)
                bbox_se_latlon = scanalyzer_to_latlon(x_s, y_e)

                # TERRA-REF
                lon_shift = 0.000020308287

                # Drone
                lat_shift = 0.000018292 #0.000015258894
                b_box =  ( bbox_se_latlon[0] - lat_shift,
                            bbox_nw_latlon[0] - lat_shift,
                            bbox_nw_latlon[1] + lon_shift,
                            bbox_se_latlon[1] + lon_shift)
                filename = os.path.basename(metadata)
            if JSON is not JSON_dict:
                JSON_dict[time, filename, gantry_x, gantry_y, gantry_z, b_box] = "Date, Time, Gantry_x, Gantry_y, Gantry_z, b_box"
            else:
                print("JSON already in Dictionary")
        return sorted(JSON_dict)
    
    def bbox_df(season, date):
        image_bbox = JSON.b_box(season, date)
        bbox_df = pd.DataFrame.from_dict(image_bbox)
        bbox_df.columns = ['Date and Time', 'Image Name', 'Gantry_x', 'Gantry_y', "Gantry_z", "b_box"]
        return bbox_df

In [45]:
image_bbox = JSON.bbox_df('season_10_yr_2020', '2020-03-03')

In [46]:
shp = gpd.read_file('/Users/sebastiancalleja/Desktop/season10_multi_latlon_geno.geojson')

In [47]:
# isolated one bbox
bbox1 = image_bbox['b_box'].loc[1]

In [49]:
# Used shapely to create a polygon out of the bbox points
#polygon1 = Polygon([[bbox1[2], bbox1[0]], [bbox1[3], bbox1[1]], [bbox1[3], bbox1[0]], [bbox1[2], bbox1[1]]])
polygon1 = Polygon([[bbox1[2], bbox1[1]], [bbox1[3], bbox1[1]], [bbox1[3], bbox1[0]], [bbox1[2], bbox1[0]]])

In [50]:
# Create polygons using the two image points
# creates df from list returned above and adds it to original df with all the information
polygon_list = []
class Get_poly:
    def to_poly (image_bbox):
        for i, row in image_bbox.iterrows():
            bbox = image_bbox['b_box'].loc[i]
            polygon = Polygon([[bbox[2], bbox[1]], [bbox[3], bbox[1]], [bbox[3], bbox[0]], [bbox[2], bbox[0]]])
            polygon_list.append(polygon)
            polygon_df = pd.DataFrame(polygon_list)
            polygon_df.rename(columns={0: 'bbox_geometry'}, inplace=True)
            image_bbox['bbox_geometry'] = polygon_df
        return image_bbox

In [52]:
Get_poly.to_poly(image_bbox)

,Date and Time,Image Name,Gantry_x,Gantry_y,Gantry_z,b_box,bbox_geometry
0,03/03/2020 08:45:33,fbf75978-ea6b-4d7c-8d85-9b21835c30fb_metadata....,207.938000,0.011,0.840,"(33.07641316860811, 33.076418830840154, -111.9...",POLYGON ((-111.9747927939666 33.07641883084015...
1,03/03/2020 08:45:35,4cb25050-63bf-4669-831f-2c6c7a5c31fa_metadata....,207.938000,0.522,0.841,"(33.07641315844808, 33.07641882742064, -111.97...",POLYGON ((-111.9747982637514 33.07641882742064...
2,03/03/2020 08:45:37,e26b63cf-265a-4235-9aed-1b0972c18fcf_metadata....,207.938000,1.011,0.841,"(33.07641315195049, 33.07641882092285, -111.97...",POLYGON ((-111.9748034952172 33.07641882092285...
3,03/03/2020 08:45:38,d71b8eef-4830-4839-94cb-a5d3e036684f_metadata....,207.938000,1.515,0.841,"(33.07641314525337, 33.07641881422551, -111.97...",POLYGON ((-111.9748088871574 33.07641881422551...
4,03/03/2020 08:45:40,f975830a-9676-49eb-a622-d2b05667ff6c_metadata....,207.938000,2.012,0.841,"(33.076413138649045, 33.07641880762096, -111.9...",POLYGON ((-111.9748142042095 33.07641880762096...
...,...,...,...,...,...,...,...
9265,03/03/2020 13:27:46,cb43835e-f50a-47d9-b3ef-6304f4d34d09_metadata....,3.800988,2.007,0.841,"(33.07457151326074, 33.074577182236254, -111.9...",POLYGON ((-111.9748135207335 33.07457718223625...
9266,03/03/2020 13:27:48,2141e0d5-7395-48d3-923b-eebe74586e1a_metadata....,3.800988,1.509,0.841,"(33.07457151987629, 33.07457718885202, -111.97...",POLYGON ((-111.9748081930941 33.07457718885202...
9267,03/03/2020 13:27:50,958fa7a8-53de-400d-b7be-2a17bd6ceadc_metadata....,3.800988,1.005,0.841,"(33.07457152657129, 33.074577195547256, -111.9...",POLYGON ((-111.9748028012663 33.07457719554726...
9268,03/03/2020 13:27:51,9bbd568e-fc9d-4508-b2aa-ff73c4aca5b6_metadata....,3.800492,0.508,0.841,"(33.07457152869842, 33.07457719767459, -111.97...",POLYGON ((-111.9747974843235 33.07457719767459...


In [68]:
# Takes bbox polygon and iterates it through plot polygons to see where they intersect
def intersection(bbox_polygon):
    intersects = bbox_polygon.intersects
    plot = None
    intersection_list = []
    for i, row in shp.iterrows():
        plot_polygon = row['geometry']
        intersection = intersects(plot_polygon)
        if intersection == True:
            plot = [row['ID']]
            intersection_list.append(plot)
    return intersection_list

In [69]:
# iterates through all of the bbox polygons and feeds them into the intersection function
image_bbox["plot"] = None
for i, row in image_bbox.iterrows():
    bbox_polygon = row['bbox_geometry']
    plot = intersection(bbox_polygon)
    image_bbox.at[i,'plot'] = plot

In [71]:
# finds which image polygons were not associated with a plot
# there were a total of 208
no_intersection = image_bbox.loc[(image_bbox['plot'].str.len() == 0),:]

In [105]:
# setting parameters to figure out why the first bbox was not associated to a plot
bbox0 = image_bbox['b_box'].loc[0]
polygon0 = Polygon([[bbox0[2], bbox0[1]], [bbox0[3], bbox0[1]], [bbox0[3], bbox0[0]], [bbox0[2], bbox0[0]]])
geometry1 = shp['geometry'].loc[1]

In [94]:
# trying to see if polygons touch rather than intersect
touches = polygon0.touches
plot = None
intersection_list = []
for i, row in shp.iterrows():
    plot_polygon = row['geometry']
    intersection = touches(plot_polygon)
    if intersection == True:
        print(intersection)
        plot = [row['ID']]
        intersection_list.append(plot)

In [131]:
# finds the number of plots each image got associated with
plot_total_list = []
for i, row in image_bbox.iterrows():
    total_plots = len(image_bbox['plot'].loc[i])
    plot_total_list.append(total_plots)

In [174]:
# adds the number of plots that an image intersected with to the main df 
total_plot_df = pd.DataFrame(plot_total_list)
total_plot_df.rename(columns={0: 'total_plots'}, inplace=True)
image_bbox['total_plots'] = total_plot_df
image_bbox['total_plots'].value_counts()

2    6196
1    1894
4     879
0     208
3      93
Name: total_plots, dtype: int64

In [162]:
# finds which image intersected with 3 plots
intersect_three = []
for i, row in image_bbox.iterrows():
    plot = row['plot']
    total_plots = row['total_plots']
    if total_plots == 3:
        intersect_three.append(plot)

In [163]:
# creates a df of the 3 plots that were intersected
intersect3_df = pd.DataFrame(intersect_three)
intersect3_df.rename(columns={0: 'plots_intersected thrice'}, inplace=True)

In [164]:
intersect3_df

,plots_intersected thrice,1,2
0,[MAC Field Scanner Season 10 Range 53 Column 11],[MAC Field Scanner Season 10 Range 53 Column 12],[MAC Field Scanner Season 10 Range 52 Column 12]
1,[MAC Field Scanner Season 10 Range 52 Column 11],[MAC Field Scanner Season 10 Range 52 Column 12],[MAC Field Scanner Season 10 Range 51 Column 11]
2,[MAC Field Scanner Season 10 Range 35 Column 30],[MAC Field Scanner Season 10 Range 34 Column 30],[MAC Field Scanner Season 10 Range 34 Column 31]
3,[MAC Field Scanner Season 10 Range 35 Column 30],[MAC Field Scanner Season 10 Range 34 Column 29],[MAC Field Scanner Season 10 Range 34 Column 30]
4,[MAC Field Scanner Season 10 Range 33 Column 35],[MAC Field Scanner Season 10 Range 32 Column 35],[MAC Field Scanner Season 10 Range 32 Column 36]
...,...,...,...
88,[MAC Field Scanner Season 10 Range 4 Column 7],[MAC Field Scanner Season 10 Range 4 Column 8],[MAC Field Scanner Season 10 Range 3 Column 7]
89,[MAC Field Scanner Season 10 Range 4 Column 6],[MAC Field Scanner Season 10 Range 3 Column 5],[MAC Field Scanner Season 10 Range 3 Column 6]
90,[MAC Field Scanner Season 10 Range 3 Column 11],[MAC Field Scanner Season 10 Range 3 Column 12],[MAC Field Scanner Season 10 Range 2 Column 11]
91,[MAC Field Scanner Season 10 Range 3 Column 8],[MAC Field Scanner Season 10 Range 2 Column 7],[MAC Field Scanner Season 10 Range 2 Column 8]
